In [168]:
from sklearn.datasets import load_iris
from scipy.special import log_softmax
import numpy
from dataclasses import dataclass

from funcs import log_density


In [169]:
load_iris().keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])

In [170]:
load_iris()["data"][0]

array([5.1, 3.5, 1.4, 0.2])

In [176]:
# Create prior weights mean
prior_weights_mean: "numpy.ndarray" = numpy.zeros(shape=(3,4))

# Weights precision
weights_precision: "float" = 1.0

In [175]:
log_density(numpy.random.normal(size=(3,4)), precision=weights_precision, data=load_iris()["data"], targets=load_iris()["target"])

-505.2739308026266

In [ ]:
def acceptance_prob():